In [11]:
import requests

class CompetitionSystem:
    def __init__(self):
        # base url
        self.__snap_url = 'http://192.168.100.1/snap'
        self.__IoT_url = "http://192.168.100.1/train"

        #teamID 任意のIDを設定します
        self.__id_parameter = 4 

    def snap(self, image_path):
        # リクエストヘッダー
        headers = {'Content-Type': 'image/png'}
        
        # 完全なURLを生成
        url = f'{self.__snap_url}?id={self.__id_parameter}'

        # PNG画像ファイルを読み取り、HTTP POSTリクエストを送信
        with open(image_path, 'rb') as file:
            response = requests.post(url, headers=headers, data=file)
        
        # レスポンスを処理する（必要に応じて）
        if response.status_code == 201:
            print('[ snap ]リクエストが成功しました')
            return True
            # レスポンスの内容を表示する場合は、以下を使用します
            # print(response.text)
        else:
            print(f'[ snap ]リクエストが失敗しました。ステータスコード: {response.status_code}')
        return False

    def IoT_SET(self, pwm_value):
        payload = {"pwm": pwm_value}
        response = requests.put(self.__IoT_url, params=payload)

        if response.status_code == 200:
            return True
        else:
            print(f'[ IoT_SET ]リクエストが失敗しました。ステータスコード: {response.status_code}')
        return False

    # データの数値変換関数
    def __convert_to_float(self, data):
        if data is None:
            raise ValueError(":各種データの情報がありません")
        try:
            return float(data)
        except ValueError:
            raise ValueError(f":{data}をFloat型に解決できません")

    def IoT_GET(self):
        # GETリクエストを送信
        response = requests.get(self.__IoT_url)

        # レスポンスが成功した場合、JSONデータを解析する
        if response.status_code == 200:
            data = response.json()

            if data is None:
                print("[ IoT_GET ]リクエストデータの情報がありません")
            else:
                try:
                    # 数値に変換
                    accel_x = self.__convert_to_float(data['accel']['x'])
                    accel_y = self.__convert_to_float(data['accel']['y'])
                    accel_z = self.__convert_to_float(data['accel']['z'])

                    gyro_x = self.__convert_to_float(data['gyro']['x'])
                    gyro_y = self.__convert_to_float(data['gyro']['y'])
                    gyro_z = self.__convert_to_float(data['gyro']['z'])

                    volt = self.__convert_to_float(data['volt'])

                    print(end=f"[Accel: X={accel_x}, Y={accel_y}, Z={accel_z}], ")
                    print(end=f"[Gyro: X={gyro_x}, Y={gyro_y}, Z={gyro_z}], ")
                    print(end=f"[Volt: {volt}]\n")

                    # 辞書型に変換
                    ret_data = {
                        'accel': {
                            'x': accel_x,
                            'y': accel_y,
                            'z': accel_z
                        },
                        'gyro': {
                            'x': gyro_x,
                            'y': gyro_y,
                            'z': gyro_z
                        },
                        'volt': volt
                    }

                    return ret_data

                except (KeyError, ValueError) as e:
                    print(f"[ IoT_GET ]処理データエラー: {e}")

        else:
            print(f"[ IoT_GET ]リクエストが失敗しました。ステータスコード: {response.status_code}")
        return False

In [12]:
import time
if __name__ == "__main__":
    competitionSystem = CompetitionSystem()
    t = 50
    competitionSystem.IoT_SET(50)
    for i in range(t):
        #competitionSystem.snap('vga.png')

        competitionSystem.IoT_GET()

        time.sleep(100/1000)
    competitionSystem.IoT_SET(0)

[ snap ]リクエストが成功しました
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
